In [1]:
import os
import re
import glob
import datetime
import numpy as np
from keras import utils, models, layers, backend, callbacks

Using TensorFlow backend.


In [2]:
CAF_DOSE = 200
STAGE = 'NREM'
TEST_SUBJECT_COUNT = 3

DATA_PATH = 'C:\\Users\\Philipp\\Documents\\Caffeine\\raw_eeg{dose}'.format(dose=CAF_DOSE)

In [3]:
class RawGenerator(utils.Sequence):
    
    def __init__(self, files_caf, files_plac, files_per_batch):
        self.files_caf, self.files_plac = np.array(files_caf), np.array(files_plac)
        self.files_per_batch = files_per_batch

    def __len__(self):
        return int(np.ceil(min(len(self.files_caf), len(self.files_plac)) / self.files_per_batch))

    def __getitem__(self, idx):
        start = idx * self.files_per_batch
        end = (idx + 1) * self.files_per_batch
        
        batch_files_caf = self.files_caf[start:end]
        batch_files_plac = self.files_plac[start:end]
        
        batch_data_list_caf = [np.load(file) for file in batch_files_caf]
        samples_per_file_caf = [x.shape[0] for x in batch_data_list_caf]
        
        batch_data_list_plac = [np.load(file) for file in batch_files_plac]
        samples_per_file_plac = [x.shape[0] for x in batch_data_list_plac]
        
        batch_x = np.concatenate(batch_data_list_caf + batch_data_list_plac, axis=0)
        batch_y = np.array([[1, 0]] * np.sum(samples_per_file_caf) + [[0, 1]] * np.sum(samples_per_file_plac))
        
        batch_x = utils.normalize(batch_x, axis=0)
        
        perm = np.random.permutation(batch_x.shape[0])
        return batch_x[perm], batch_y[perm]
    
    def on_epoch_end(self):
        self.files_caf = self.files_caf[np.random.permutation(len(self.files_caf))]
        self.files_plac = self.files_plac[np.random.permutation(len(self.files_plac))]

In [ ]:
def get_subject_id(path):
    return re.match('\S\d+', path.split(os.sep)[-1].split('_')[0])[0]
    
caf_files = glob.glob(os.path.join(DATA_PATH, f'*{STAGE}*CAF*'))
plac_files = glob.glob(os.path.join(DATA_PATH, f'*{STAGE}*PLAC*'))

caf_subjects = set([get_subject_id(file) for file in caf_files])
plac_subjects = set([get_subject_id(file) for file in plac_files])

subjects = caf_subjects & plac_subjects

test_subjects = set()
while len(test_subjects) < TEST_SUBJECT_COUNT:
    test_subjects.add(np.random.choice(list(subjects)))
    
caf_files_train = [file for file in caf_files if get_subject_id(file) not in test_subjects]
plac_files_train = [file for file in plac_files if get_subject_id(file) not in test_subjects]
caf_files_test = [file for file in caf_files if get_subject_id(file) in test_subjects]
plac_files_test = [file for file in plac_files if get_subject_id(file) in test_subjects]

train = RawGenerator(caf_files_train, plac_files_train, 3)
test = RawGenerator(caf_files_test, plac_files_test, 3)

In [ ]:
backend.clear_session()

model = models.Sequential()
model.add(layers.LSTM(units=32, input_shape=(5120, 20)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                6784      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 7,906
Trainable params: 7,906
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
name = datetime.datetime.now().strftime('lstm_%Y-%m-%d_%H-%M-%S')
tensorboard = callbacks.TensorBoard(log_dir='..\\results\\logs\\' + name)

model.fit_generator(generator=train,
                    epochs=100,
                    validation_data=test,
                    max_queue_size=6,
                    workers=6,
                    callbacks=[tensorboard])

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
108/108 [==============================] - 1360s 13s/step - loss: 0.6933 - acc: 0.5032 - val_loss: 0.6930 - val_acc: 0.5054
Epoch 2/100
 40/108 [==========>...................] - ETA: 20:31 - loss: 0.6930 - acc: 0.5035

In [ ]:
print('Training accuracy:', model.evaluate_generator(train)[1])
print('Testing accuracy: ', model.evaluate_generator(test)[1])